In [ ]:
# https://www.kaggle.com/datasets/kartik2112/fraud-detection/code

# O atributo de interesse é is_fraud

import pandas as pd

transacoes_train_df = pd.read_csv('fraudTrain.csv')
transacoes_test_df = pd.read_csv('fraudTest.csv')


In [ ]:
transacoes_train_df.info()

In [ ]:
# Questao 1
# Caso a regra de decisão da iComida seja de que toda transação é aceita
# e nunca é bloqueada por fraude, qual seria a acurácia deste "modelo"
# nas transações de teste?
# dica: use https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.value_counts.html
# para contar frequências.

transacoes_test_df['is_fraud'].value_counts()


In [ ]:

#acuracia = (1289169) / (1289169 + 7506)
#acuracia

acuracia = (553574) / (553574 + 2145)
acuracia

In [ ]:
# Questão 3
# Antes de estudar aprendizado de máquina na Faculdade XP,
# sua alternativa para criar modelos preditivos era usar sua intuição, experiência
# e olhar para os dados em busca de padrões.
# Por exemplo, sua experiência lhe diz que transações fraudulentas tendem a
# movimentar valores mais altos, e você confirma isto pelo gráfico abaixo.

import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

plt.figure(figsize=(10, 6))
sns.kdeplot(data=transacoes_train_df, x='amt', hue='is_fraud', fill=True, common_norm=False)

plt.title('Distribuição de probabilidade para fraude/não-fraude', fontsize=14)
plt.xlabel('Amount (amt)', fontsize=12)
plt.ylabel('Densidade', fontsize=12)
plt.xlim(0, 1500)

# Exibindo o gráfico
plt.show()

In [ ]:
# Questão 3.
# Então, você tenta criar uma classificador "na mão".
# Você monta classificadores em que, a partir de um certo valor
# valor_minimo_fraude, a transação será classificada como fraude.

import numpy as np
from sklearn.base import BaseEstimator, ClassifierMixin

class ClassificadorDeFraudeManual(BaseEstimator, ClassifierMixin):
    def fit(self, X, y):
        return self

    def predict(self, X):

       def predict_fraud(x):
            valor_minimo_fraude = 1000
            if x['amt'] > valor_minimo_fraude:
                return 1
            else:
                return 0

       return X.apply(predict_fraud, axis=1)


In [ ]:
X = transacoes_train_df[['amt']]
Y = transacoes_train_df['is_fraud']


In [ ]:
# Questao 3.
# Você pretende criar uma classificador manual para ajudar o time anti-fraude do iComida.
# A ideia é que, ao invés de ter que examinar dezenas de milhares de potenciais fraudes por dia,
# o time examine apenas um sub-conjunto de potenciais fraudes que seu algoritmo identificou.
# Você experimenta com 4 limiares: 200, 400, 800 e 1.000 reais.
# Qual destes valores de corte gera menos falsos positivos, ou seja, traz mais confiança
# para o time de fraude de que as transações indicadas pelo seu sistema são realmente fraudes?

# Questão 4
# Caso você queira que o seu sistema deixe menos fraudes sem serem escrutinadas pelo time de fraude,
# mesmo que eles tenham que examinar muitas transações legítimas, qual limiar você escolheria?

from sklearn.metrics import classification_report

classificador_manual = ClassificadorDeFraudeManual()
classificador_manual.fit(X, Y)

Y_pred = classificador_manual.predict(X)

report = classification_report(Y, Y_pred)
print(report)


In [ ]:
# Questão 5.
# A partir dos seus estudos em aprendizado de máquina, você
# decide usar os dados para criar o programa que decide se uma transação é fraude,
# ao invés de contar apenas com sua intuição e capacidade de analisar gráficos.
# Você pediu pro chatGPT gerar uma árvode de decisão que classifica fraudes
# a partir do montante (amt) da transação, e ele gerou o seguinte código.
# Incrivelmente, a precisão é 98%! De cada 100 transações que o modelo acusa
# como fraude, ele se equivoca em apenas 2 casos, em média.

from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier

decision_tree_model = DecisionTreeClassifier(random_state=42)
decision_tree_model.fit(X, Y)

Y_pred = decision_tree_model.predict(X)

report = classification_report(Y, Y_pred)
print(report)

In [ ]:
# Questão 5. Animado com a capacidade de modelos de aprendizado de máquina, você
# quer entender mais sobre o modelo aprendido.
# Visite https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier.get_depth
# e responde qual é a profundidade da árvore aprendida.

decision_tree_model.get_depth()

In [ ]:
# questao 6
# Você acha a profundidade da árvore muito alta, afinal, você está criando a árvore com apenas um atributo (amt)!
# Você usa https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier.get_n_leaves para descobrir quantas “folhas” tem a árvore.
# Um nó “folha”, em uma árvore, são os nós finais que efetivamente classificam uma transação entre fraude e legítima.
# Qual o número de folhas da árvore que você criou?

decision_tree_model.get_n_leaves()

In [ ]:
# questao 7
# Você decide, agora, computar a precisão e cobertura do modelo no conjunto de treino e de teste.
# O que você observa?

X_train = transacoes_train_df[['amt']]
y_train = transacoes_train_df['is_fraud']

X_test = transacoes_test_df[['amt']]
y_test = transacoes_test_df['is_fraud']


In [ ]:
y_pred_test = decision_tree_model.predict(X_test)

report = classification_report(y_test, y_pred_test)
print(report)

In [ ]:
# Questao 9
# Você percebe que precisa realizar validação cruzada para avaliar o modelo de forma justa.
# O modelo que maximiza a cobertura tem qual profundidade?

from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Definir os parâmetros para o GridSearch
param_grid = {
    # complete aqui
    'max_depth': [1, 2, 3, 4, 5, 6, 10, 90],
}

dt_classifier = DecisionTreeClassifier(random_state=42)
grid_search = GridSearchCV(estimator=dt_classifier, param_grid=param_grid, cv=3, scoring='recall')

grid_search.fit(X_train, y_train)
best_model = grid_search.best_estimator_

y_pred_test = best_model.predict(X_test)

report_test = classification_report(y_test, y_pred_test)
print(report_test)

In [ ]:
# Questao 9
# Imprimir profundidade máxima e número de nós
print(f"Profundidade máxima da árvore: {best_model.tree_.max_depth}")
print(f"Número de nós na árvore: {best_model.tree_.node_count}")

In [ ]:
# Questao 9
grid_search.cv_results_

In [ ]:
# Questao 10 / 11
# Agora você quer melhorar o modelo adicionando um novo atributo: a hora do dia em que a transação aconteceu.
# ALTERE o código que você já tem neste notebook para adicionar a variável 'Hour' ao modelo.
# O que aconteceu?

transacoes_train_df['trans_date_trans_time'] = pd.to_datetime(transacoes_train_df['trans_date_trans_time'])
transacoes_train_df['Hour'] = transacoes_train_df['trans_date_trans_time'].dt.hour

transacoes_test_df['trans_date_trans_time'] = pd.to_datetime(transacoes_test_df['trans_date_trans_time'])
transacoes_test_df['Hour'] = transacoes_test_df['trans_date_trans_time'].dt.hour

X_train = transacoes_train_df[['amt', 'Hour']]
y_train = transacoes_train_df['is_fraud']

X_test = transacoes_test_df[['amt', 'Hour']]
y_test = transacoes_test_df['is_fraud']


In [ ]:
# Questao 10 / 11
# Agora você quer melhorar o modelo adicionando um novo atributo: a hora do dia em que a transação aconteceu.
# ALTERE o código que você já tem neste notebook para adicionar a variável 'Hour' ao modelo.
# O que aconteceu?

from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Definir os parâmetros para o GridSearch
param_grid = {
    # complete aqui
    'max_depth': [1, 2, 3, 4, 5, 6, 10, 90],
}

dt_classifier = DecisionTreeClassifier(random_state=42)
grid_search = GridSearchCV(estimator=dt_classifier, param_grid=param_grid, cv=3, scoring='precision')

grid_search.fit(X_train, y_train)
best_model = grid_search.best_estimator_

y_pred_test = best_model.predict(X_test)

report_test = classification_report(y_test, y_pred_test)
print(report_test)

In [ ]:
# Questao 12
#Troque a chamada de GridSearchCV para otimizar a cobertura (recall), ao invés de otimizar a precisão (recall).
#Quais os novos valores de precisão e cobertura?

from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Definir os parâmetros para o GridSearch
param_grid = {
    # complete aqui
    'max_depth': [1, 2, 3, 4, 5, 6, 10, 90],
}

dt_classifier = DecisionTreeClassifier(random_state=42)
grid_search = GridSearchCV(estimator=dt_classifier, param_grid=param_grid, cv=3, scoring='recall')

grid_search.fit(X_train, y_train)
best_model = grid_search.best_estimator_

y_pred_test = best_model.predict(X_test)

report_test = classification_report(y_test, y_pred_test)
print(report_test)